In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point, LineString
from shapely.ops import split

In [3]:
#Read in routes feature class as shapefile
gdfRoutes = gpd.read_file('..\\data\\processed\\Routes.shp')

In [4]:
#Columns to drop to keep things tidy
drop_cols = ['Facility N', 'Address', 'City', 'County Nam',
             'Zip', 'Latitude', 'Longitude', 'Regulated', 
             'Allowable', 'Total Wast']

In [6]:
#Copy routes geodataframe and update geometry to start points
gdfStart = gdfRoutes.copy(deep=True)
gdfStart['geometry'] = gdfRoutes['geometry'].apply(lambda x: Point(x.coords[0]))
gdfStart.drop(columns=drop_cols,axis=1,inplace=True)
gdfStart.to_file('../scratch/starts.shp')

In [7]:
#Copy routes geodataframe and update geometry to start points
gdfEnd = gdfRoutes.copy(deep=True)
gdfEnd['geometry'] = gdfRoutes['geometry'].apply(lambda x: Point(x.coords[-1]))
gdfEnd.drop(columns=drop_cols,axis=1,inplace=True)
gdfEnd.to_file('../scratch/ends.shp')

Create junctions along route segments where upstream routes join them - done by iterating through each route feature and splitting it by the set of end points. 

In [8]:
#Combine endpoints into a single multipoint object
ends = gdfEnd.geometry.unary_union

In [9]:
#Create a geoseries of split routes (geometry collections)
theSplits = gdfRoutes.geometry.apply(lambda x: split(x,ends))

* Create a feature class of all segments (routes split at junctions)

In [10]:
#Create lists to fill
links = []
startIDs = []; endIDs = []
biogas = []
geom = []
#Iterate and add items to the list
for index, row in gdfRoutes.iterrows():
    #Iterate through split segments in the geometry collection
    for line in theSplits[index].geoms:
        #Add items to the list
        links.append(row['index'])
        biogas.append(row['Biogas P_1'])
        geom.append(line)

#Construct an output geodataframe
gdfSegments = gpd.GeoDataFrame(pd.DataFrame({'route_id':links}),
                               geometry = geom, crs = gdfRoutes.crs)

#Add the index as a unique segment ID  
gdfSegments['edge_ID'] = gdfSegments.index

* Create feature classes from from and to nodes

In [11]:
#Construct a gdf of segement start and end points
gdfFromNodes = gdfSegments.copy(deep=True)
gdfFromNodes['geometry'] = gdfFromNodes['geometry'].apply(lambda x: Point(x.coords[0]))
gdfFromNodes['from_id'] = gdfFromNodes.index

gdfToNodes = gdfSegments.copy(deep=True)
gdfToNodes['geometry'] = gdfToNodes['geometry'].apply(lambda x: Point(x.coords[-1]))
gdfToNodes['to_id'] = gdfToNodes.index

* Join the from and to node IDs to each segment

In [12]:
#Find the start point corresponding with this value
def getNodeID(line_segment,start=True):
    #Get the start or end point
    if start: 
        gdf = gdfFromNodes #Set the search geodataframe as the one of start points
        idx = 0        #Set the index to the first point of the line segment
        col = 'from_id'
    else:
        gdf = gdfToNodes   #Set the search geodataframe as the one of end points
        idx = -1       #Set the index to the last point of the line segment
        col = 'to_id'
    #Create the search point object, buffered 5 meters
    theSearchPoly = Point(line_segment.coords[idx]).buffer(5)
    #Select all records found within the buffered search points
    gdfSelect = gdf[gdf.geometry.within(theSearchPoly)]
    #Return the index value of the returned record
    if gdfSelect.shape[0]>0: return gdfSelect[col].values[0]
    else: return -1

In [13]:
#Add start node ids to edges
gdfSegments['FromID'] = gdfSegments['geometry'].apply(getNodeID)
#Add end node ids to edges
gdfSegments['ToID'] = gdfSegments['geometry'].apply(lambda x: getNodeID(x,start=False))

In [16]:
#Write to file
gdfSegments.to_file('../scratch/LineSegements.shp')
gdfFromNodes.to_file('../scratch/FromNodes.shp')
gdfToNodes.to_file('../scratch/ToNodes.shp')

We want to build an edge list such that each route segment includes it's from node ID, to node ID, and the biogas associated with the from node id

In [ ]:
#Spatially join biogas potential to start nodes
gdf_StartNodes = gpd.sjoin(left_df=gdfSegment_starts,right_df=gdfStart,how='left')
gdf_StartNodes.drop(columns=['index_right','level_0','index'],axis=1,inplace=True)
gdf_StartNodes.fillna(0,inplace=True)
gdf_StartNodes.head()

In [ ]:
#Spatially join edge IDs to end nodes
gdf_StartNodes = gpd.sjoin(left_df=gdf_StartNodes,right_df=gdfEnd[['index','geometry']],how='left')
#gdf_StartNodes.drop(columns=['index_right','index'],axis=1,inplace=True)
gdf_StartNodes.fillna(0,inplace=True)
gdf_StartNodes.head()

In [ ]:
#Add from node and to node to gdfSegments
gdfSegments2 = gpd.sjoin(left_df=gdfSegments,
                         right_df=gdfStart[['index','geometry']],
                         how='left',rsuffix='start')

In [ ]:
#Get the start point of the line
line.coords[0]

In [ ]:
gdfSegments3 = gpd.sjoin(left_df=gdfSegments2,
                         right_df=gdfEnd[['index','geometry']],
                         how='left',rsuffix='end')

In [ ]:
gdfSegments3.to_file('../data/processed/LineSegements_StartEnd.shp')

In [ ]:
gdfRoutes.columns